In [ ]:
!pip install --upgrade albumentations



In [ ]:
import albumentations as A
import cv2
print("Albumentations and OpenCV are working ✅")


In [ ]:
# For augmentation of delam images 
import albumentations as A
import cv2
import os
import re
import numpy as np

# --- Augmentation setup ---
def get_transform():
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.Blur(blur_limit=3, p=0.2)
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# --- Paths ---
image_dir = r"C:/Users/flexi/DeepL/New_Detection1/images/train"
label_dir = r"C:/Users/flexi/DeepL/New_Detection1/labels/train"
aug_img_dir = r"C:/Users/flexi/DeepL/New_Aug/aug_images"
aug_lbl_dir = r"C:/Users/flexi/DeepL/New_Aug/aug_labels"

os.makedirs(aug_img_dir, exist_ok=True)
os.makedirs(aug_lbl_dir, exist_ok=True)

# --- Delamination filter ---
def find_d_images(directory):
    pattern = re.compile(r".*_D\d+\.(?:jpg|jpeg|png|bmp|gif|webp)$", re.IGNORECASE)
    return [fname for fname in os.listdir(directory) if pattern.search(fname.strip())]

delam_images = find_d_images(image_dir)

print(f"\n[INFO] Found {len(delam_images)} matching delamination images:")
for img in sorted(delam_images):
    print(f" - {img}")

augmented_count = 0

# --- Augmentation loop ---
for fname in delam_images:
    img_path = os.path.join(image_dir, fname)
    label_path = os.path.join(label_dir, os.path.splitext(fname)[0] + ".txt")

    if not os.path.exists(label_path):
        print(f"[!] Label missing for {fname}, skipping.")
        continue

    image = cv2.imread(img_path)
    if image is None:
        print(f"[!] Failed to load {fname}, skipping.")
        continue

    h, w, _ = image.shape

    # Load YOLO-format bounding boxes with validation
    bboxes, classes = [], []
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                print(f"[!] Invalid label in {fname}, skipping line.")
                continue

            cls, x, y, bw, bh = map(float, parts)
            x = np.clip(x, 0.0, 1.0)
            y = np.clip(y, 0.0, 1.0)
            bw = np.clip(bw, 0.0, 1.0 - x)
            bh = np.clip(bh, 0.0, 1.0 - y)
            bboxes.append([x, y, bw, bh])
            classes.append(int(cls))

    if not bboxes:
        print(f"[!] No valid bounding boxes in {fname}, skipping.")
        continue

    # Generate two augmented versions
    for i in range(2):
        try:
            transformed = get_transform()(image=image, bboxes=bboxes, class_labels=classes)
            aug_image = transformed['image']
            aug_bboxes = transformed['bboxes']
            aug_classes = transformed['class_labels']

            if not aug_bboxes:
                print(f"[!] No bboxes after augmentation {i+1} for {fname}, skipping.")
                continue

            # Save augmented image and label
            base_name = os.path.splitext(fname)[0] + f"_aug{i+1}"
            out_img_path = os.path.join(aug_img_dir, base_name + ".jpg")
            out_lbl_path = os.path.join(aug_lbl_dir, base_name + ".txt")

            cv2.imwrite(out_img_path, aug_image)

            with open(out_lbl_path, 'w') as f:
                for cls, bbox in zip(aug_classes, aug_bboxes):
                    x, y, bw, bh = bbox
                    x = max(0.0, min(1.0, x))
                    y = max(0.0, min(1.0, y))
                    bw = max(0.0, min(1.0 - x, bw))
                    bh = max(0.0, min(1.0 - y, bh))
                    if bw < 0.01 or bh < 0.01:
                        print(f"[!] Skipping very small box in {fname} aug{i+1}")
                        continue
                    f.write(f"{cls} {x:.6f} {y:.6f} {bw:.6f} {bh:.6f}\n")

            print(f"[✓] Augmented: {base_name}.jpg, Labels: {base_name}.txt")
            augmented_count += 1

        except Exception as e:
            print(f"[X] Error augmenting {fname} (aug{i+1}): {str(e)}")

print(f"\n[INFO] Total delamination images augmented: {augmented_count}")


In [ ]:
# To access delam images count
import os
import re

def find_d_images(directory):
    """
    Find all image files with pattern _D followed by digits and image extension.
    
    Args:
        directory: Path to search for images
        
    Returns:
        List of matching filenames
    """
    # More comprehensive pattern that:
    # 1. Allows optional prefixes/suffixes
    # 2. Handles multiple image extensions
    # 3. Is case insensitive
    # 4. Accounts for possible .jpg variations
    pattern = re.compile(
       r".*_C\d+.*\.(?:jpg|jpeg|png|bmp|gif|webp)$", 
        re.IGNORECASE
    )
    
    matched_images = []
    for fname in os.listdir(directory):
        if pattern.search(fname.strip()):
            matched_images.append(fname)
    
    return matched_images

# Example usage
image_dir = r"C:/Users/flexi/DeepL/processed_dataset/images"
matches = find_d_images(image_dir)

print(f"Found {len(matches)} matching images:")
for img in sorted(matches):
    print(f" - {img}")

In [ ]:
!pip install numpy==2.0.0

In [ ]:
#to verify annotations with original image compaison but doesnt save
import cv2
import os
import matplotlib.pyplot as plt
import re

def load_image_and_labels(img_path, label_path):
    image = cv2.imread(img_path)
    bboxes, classes = [], []
    if not os.path.exists(label_path) or image is None:
        return image, bboxes, classes

    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 5:
                cls, x, y, bw, bh = map(float, parts)
                bboxes.append([x, y, bw, bh])
                classes.append(int(cls))
    return image, bboxes, classes

def draw_yolo_bboxes(image, bboxes, labels=None, color=(0, 255, 0)):
    if image is None:
        return None
    h, w = image.shape[:2]
    img = image.copy()
    for i, (x, y, bw, bh) in enumerate(bboxes):
        x1 = int((x - bw / 2) * w)
        y1 = int((y - bh / 2) * h)
        x2 = int((x + bw / 2) * w)
        y2 = int((y + bh / 2) * h)
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        if labels:
            cv2.putText(img, str(labels[i]), (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    return img

def show_images_with_annotations(base_filename, orig_img_dir, orig_lbl_dir, aug_img_dir, aug_lbl_dir):
    paths = {
        "Original": (os.path.join(orig_img_dir, base_filename + ".jpg"),
                     os.path.join(orig_lbl_dir, base_filename + ".txt")),
        "Aug1": (os.path.join(aug_img_dir, base_filename + "_aug1.jpg"),
                 os.path.join(aug_lbl_dir, base_filename + "_aug1.txt")),
        "Aug2": (os.path.join(aug_img_dir, base_filename + "_aug2.jpg"),
                 os.path.join(aug_lbl_dir, base_filename + "_aug2.txt")),
    }

    images = []
    titles = []

    for title, (img_path, lbl_path) in paths.items():
        img, bboxes, classes = load_image_and_labels(img_path, lbl_path)
        if img is not None:
            color = (0, 255, 0) if title == "Original" else (255, 0, 0) if title == "Aug1" else (0, 0, 255)
            drawn = draw_yolo_bboxes(img, bboxes, classes, color=color)
            images.append(drawn)
            titles.append(f"{base_filename}{'_aug1' if title == 'Aug1' else '_aug2' if title == 'Aug2' else ''}.jpg")

    if images:
        fig, axs = plt.subplots(1, len(images), figsize=(6 * len(images), 6))
        if len(images) == 1:
            axs = [axs]
        for ax, img, title in zip(axs, images, titles):
            ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            ax.set_title(title)
            ax.axis('off')
        plt.tight_layout()
        plt.show()
    else:
        print(f"[!] No valid images found for {base_filename}")

def find_delamination_images(folder):
    pattern = re.compile(r".*_D\d+\.(jpg|jpeg|png|bmp)$", re.IGNORECASE)
    return [f for f in os.listdir(folder) if pattern.match(f)]

# --- Update these paths ---
orig_img_dir = r"C:/Users/flexi/DeepL/New_Detection1/images/train"
orig_lbl_dir = r"C:/Users/flexi/DeepL/New_Detection1/labels/train"
aug_img_dir = r"C:/Users/flexi/DeepL/New_Aug/aug_images"
aug_lbl_dir = r"C:/Users/flexi/DeepL/New_Aug/aug_labels"

# --- Loop through all delamination images ---
delam_images = find_delamination_images(orig_img_dir)

print(f"[INFO] Found {len(delam_images)} delamination images to visualize...\n")

for fname in sorted(delam_images):
    base_name = os.path.splitext(fname)[0]
    print(f"\n📷 Showing: {base_name}")
    show_images_with_annotations(base_name, orig_img_dir, orig_lbl_dir, aug_img_dir, aug_lbl_dir)


In [ ]:
#to augment 92 images fpor crack too
import albumentations as A
import cv2
import os
import re
import numpy as np

# --- Augmentation setup ---
def get_transform():
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.Blur(blur_limit=3, p=0.2)
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# --- Paths ---
image_dir = r"C:/Users/flexi/DeepL/New_Detection1/images/train"
label_dir = r"C:/Users/flexi/DeepL/New_Detection1/labels/train"
aug_img_dir = r"C:/Users/flexi/DeepL/New_Aug_C/aug_images"
aug_lbl_dir = r"C:/Users/flexi/DeepL/New_Aug_C/aug_labels"

os.makedirs(aug_img_dir, exist_ok=True)
os.makedirs(aug_lbl_dir, exist_ok=True)

# --- Crack filter for "_C" ---
def find_c_images(directory):
    pattern = re.compile(r".*_C\d+\.(?:jpg|jpeg|png|bmp|gif|webp)$", re.IGNORECASE)
    return sorted([fname for fname in os.listdir(directory) if pattern.search(fname.strip())])

crack_images = find_c_images(image_dir)[:100]

print(f"\n[INFO] Found {len(crack_images)} crack images:")
for img in crack_images:
    print(f" - {img}")

augmented_count = 0

for fname in crack_images:
    img_path = os.path.join(image_dir, fname)
    label_path = os.path.join(label_dir, os.path.splitext(fname)[0] + ".txt")

    if not os.path.exists(label_path):
        print(f"[!] Label missing for {fname}, skipping.")
        continue

    image = cv2.imread(img_path)
    if image is None:
        print(f"[!] Failed to load {fname}, skipping.")
        continue

    h, w, _ = image.shape

    # Load YOLO-format bounding boxes with validation
    bboxes, classes = [], []
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                print(f"[!] Invalid label in {fname}, skipping line.")
                continue

            cls, x, y, bw, bh = map(float, parts)
            x = np.clip(x, 0.0, 1.0)
            y = np.clip(y, 0.0, 1.0)
            bw = np.clip(bw, 0.0, 1.0 - x)
            bh = np.clip(bh, 0.0, 1.0 - y)
            bboxes.append([x, y, bw, bh])
            classes.append(int(cls))

    if not bboxes:
        print(f"[!] No valid bounding boxes in {fname}, skipping.")
        continue

    try:
        transformed = get_transform()(image=image, bboxes=bboxes, class_labels=classes)
        aug_image = transformed['image']
        aug_bboxes = transformed['bboxes']
        aug_classes = transformed['class_labels']

        if not aug_bboxes:
            print(f"[!] No bboxes after augmentation for {fname}, skipping.")
            continue

        base_name = os.path.splitext(fname)[0] + f"_aug"
        out_img_path = os.path.join(aug_img_dir, base_name + ".jpg")
        out_lbl_path = os.path.join(aug_lbl_dir, base_name + ".txt")

        cv2.imwrite(out_img_path, aug_image)

        with open(out_lbl_path, 'w') as f:
            for cls, bbox in zip(aug_classes, aug_bboxes):
                x, y, bw, bh = bbox
                x = max(0.0, min(1.0, x))
                y = max(0.0, min(1.0, y))
                bw = max(0.0, min(1.0 - x, bw))
                bh = max(0.0, min(1.0 - y, bh))
                if bw < 0.01 or bh < 0.01:
                    print(f"[!] Skipping very small box in {fname} aug")
                    continue
                f.write(f"{cls} {x:.6f} {y:.6f} {bw:.6f} {bh:.6f}\n")

        print(f"[✓] Augmented: {base_name}.jpg, Labels: {base_name}.txt")
        augmented_count += 1

    except Exception as e:
        print(f"[X] Error augmenting {fname}: {str(e)}")

print(f"\n[INFO] Total crack images augmented: {augmented_count}")



In [ ]:
#image size check
import os
import cv2
from collections import Counter

# --- Set your image directory ---
image_dir = r"C:/Users/flexi/DeepL/New_Detection1/images/train"  # update if needed

# --- Supported image extensions ---
image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".webp", ".tiff"}

# --- Initialize variables ---
size_counter = Counter()
widths, heights = [], []
total_images = 0

# --- Walk through the directory ---
for fname in os.listdir(image_dir):
    ext = os.path.splitext(fname)[1].lower()
    if ext not in image_extensions:
        continue

    path = os.path.join(image_dir, fname)
    image = cv2.imread(path)

    if image is None:
        print(f"[!] Could not read: {fname}")
        continue

    h, w = image.shape[:2]
    size_counter[(w, h)] += 1
    widths.append(w)
    heights.append(h)
    total_images += 1

# --- Results ---
print("\n📏 Image Size Analysis")
print("=" * 30)
print(f"Total images analyzed: {total_images}\n")

print("🧮 Size Frequency:")
for size, count in size_counter.most_common():
    print(f" - {size[0]}x{size[1]} : {count} image(s)")

if widths and heights:
    print(f"\n🔍 Minimum Width: {min(widths)} px")
    print(f"🔍 Maximum Width: {max(widths)} px")
    print(f"🔍 Minimum Height: {min(heights)} px")
    print(f"🔍 Maximum Height: {max(heights)} px")


In [ ]:
#resize and padding with annotation
import os
import cv2
import shutil

# 🗂️ Paths
input_image_dir = r'C:/Users/flexi/DeepL/New_Detection1/images/train'
input_label_dir = r'C:/Users/flexi/DeepL/New_Detection1/labels/train'

output_image_dir = r'C:/Users/flexi/DeepL/New_Detection1/processed_dataset/images'
output_label_dir = r'C:/Users/flexi/DeepL/New_Detection1/processed_dataset/labels'

os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

TARGET_SIZE = 2048  # Square dimension
PADDING_COLOR = (0, 0, 0)  # Black

def resize_and_pad_image(image):
    h, w = image.shape[:2]

    # Resize if either dim > 2048
    scale = min(TARGET_SIZE / w, TARGET_SIZE / h)
    resized = cv2.resize(image, (int(w * scale), int(h * scale)))

    new_h, new_w = resized.shape[:2]
    pad_top = (TARGET_SIZE - new_h) // 2
    pad_bottom = TARGET_SIZE - new_h - pad_top
    pad_left = (TARGET_SIZE - new_w) // 2
    pad_right = TARGET_SIZE - new_w - pad_left

    padded = cv2.copyMakeBorder(resized, pad_top, pad_bottom, pad_left, pad_right,
                                borderType=cv2.BORDER_CONSTANT, value=PADDING_COLOR)
    
    return padded, scale, pad_left, pad_top, w, h

def adjust_yolo_labels(label_path, output_path, scale, pad_left, pad_top, orig_w, orig_h):
    if not os.path.exists(label_path):
        return

    with open(label_path, 'r') as f:
        lines = f.readlines()

    updated_lines = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            continue

        cls, x_center, y_center, box_w, box_h = map(float, parts)

        # Convert from relative to absolute
        abs_x = x_center * orig_w
        abs_y = y_center * orig_h
        abs_w = box_w * orig_w
        abs_h = box_h * orig_h

        # Scale and pad
        abs_x = abs_x * scale + pad_left
        abs_y = abs_y * scale + pad_top
        abs_w *= scale
        abs_h *= scale

        # Convert back to relative (new size = 2048)
        rel_x = abs_x / TARGET_SIZE
        rel_y = abs_y / TARGET_SIZE
        rel_w = abs_w / TARGET_SIZE
        rel_h = abs_h / TARGET_SIZE

        updated_line = f"{int(cls)} {rel_x:.6f} {rel_y:.6f} {rel_w:.6f} {rel_h:.6f}"
        updated_lines.append(updated_line)

    with open(output_path, 'w') as f:
        f.write('\n'.join(updated_lines))


# 🚀 Process images
image_extensions = ['.jpg', '.jpeg', '.png']

for filename in os.listdir(input_image_dir):
    if not any(filename.lower().endswith(ext) for ext in image_extensions):
        continue

    image_path = os.path.join(input_image_dir, filename)
    label_path = os.path.join(input_label_dir, os.path.splitext(filename)[0] + '.txt')

    img = cv2.imread(image_path)
    padded_img, scale, pad_left, pad_top, orig_w, orig_h = resize_and_pad_image(img)

    out_img_path = os.path.join(output_image_dir, filename)
    out_label_path = os.path.join(output_label_dir, os.path.splitext(filename)[0] + '.txt')

    cv2.imwrite(out_img_path, padded_img)
    adjust_yolo_labels(label_path, out_label_path, scale, pad_left, pad_top, orig_w, orig_h)

print("✅ Done! All images and labels saved in 'processed_dataset/'")


In [ ]:
#generating annotated images to verify
import os
import cv2

# Paths
image_dir = r"C:\Users\flexi\DeepL\processed_dataset\images"
label_dir = r"C:\Users\flexi\DeepL\processed_dataset\labels"
output_dir = r"C:\Users\flexi\DeepL\processed_dataset\annotated_images"

# Make output dir if not exists
os.makedirs(output_dir, exist_ok=True)

# Class names (change if needed)
class_names = ['Delamination', 'Crack']

# Color mapping for classes
class_colors = {
    0: (0, 255, 0),  # Green for Crack
    1: (0, 0, 255)   # Red for Delamination
}

# Iterate over images
for image_name in os.listdir(image_dir):
    if not image_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.webp')):
        continue

    # Load image
    image_path = os.path.join(image_dir, image_name)
    img = cv2.imread(image_path)
    if img is None:
        print(f"⚠️ Couldn't read image: {image_name}")
        continue

    h, w = img.shape[:2]

    # Corresponding label
    label_name = os.path.splitext(image_name)[0] + '.txt'
    label_path = os.path.join(label_dir, label_name)

    if not os.path.exists(label_path):
        print(f"❌ Missing label for: {image_name}")
        continue

    # Read label file
    with open(label_path, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split()
            if len(parts) != 5:
                print(f"⚠️ Invalid label format in {label_name}")
                continue
            cls_id, xc, yc, bw, bh = map(float, parts)
            cls_id = int(cls_id)

            # Convert YOLO format to pixel coords
            x1 = int((xc - bw / 2) * w)
            y1 = int((yc - bh / 2) * h)
            x2 = int((xc + bw / 2) * w)
            y2 = int((yc + bh / 2) * h)

            # Draw bounding box
            color = class_colors.get(cls_id, (255, 255, 255))
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

            # Add class label
            label_text = class_names[cls_id] if cls_id < len(class_names) else f'Class {cls_id}'
            cv2.putText(img, label_text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # Save annotated image
    out_path = os.path.join(output_dir, image_name)
    cv2.imwrite(out_path, img)

print("\n✅ All images annotated and saved!")
print(f"Check folder: {output_dir}")


In [25]:
import os
import shutil
import re
import random
from collections import defaultdict

def find_class_images(directory, class_tag):
    """
    Find all image files for a specific class (Crack or Delamination)
    """
    pattern = re.compile(
        rf".*_{class_tag}\d+.*\.(?:jpg|jpeg|png|bmp|gif|webp)$", 
        re.IGNORECASE
    )
    return [f for f in os.listdir(directory) if pattern.search(f)]

def separate_original_augmented(image_list):
    """
    Separate original images from augmented images based on the specific naming pattern
    """
    original = []
    augmented = []
    
    for img in image_list:
        # Original images end with _C# or _D# without any suffix
        if re.search(r'_[CD]\d+\.', img):  # Ends with _C1.jpg or _D31.jpg
            original.append(img)
        # Augmented images have _aug or _aug# after the class number
        elif re.search(r'_[CD]\d+_aug\d*\.', img):  # Matches _C1_aug.jpg or _D31_aug2.jpg
            augmented.append(img)
    
    return original, augmented

def split_dataset(original, augmented, train_orig, train_aug, val_orig, val_aug, test_orig, test_aug):
    """
    Split dataset into train, val, test with specified counts for original and augmented
    """
    # Verify we have enough images
    assert len(original) >= train_orig + val_orig + test_orig, "Not enough original images"
    assert len(augmented) >= train_aug + val_aug + test_aug, "Not enough augmented images"
    
    # Shuffle both lists
    random.shuffle(original)
    random.shuffle(augmented)
    
    # Split original images
    orig_train = original[:train_orig]
    orig_val = original[train_orig:train_orig+val_orig]
    orig_test = original[train_orig+val_orig:train_orig+val_orig+test_orig]
    
    # Split augmented images
    aug_train = augmented[:train_aug]
    aug_val = augmented[train_aug:train_aug+val_aug]
    aug_test = augmented[train_aug+val_aug:train_aug+val_aug+test_aug]
    
    # Combine original and augmented for each split
    train_files = orig_train + aug_train
    val_files = orig_val + aug_val
    test_files = orig_test + aug_test
    
    return train_files, val_files, test_files

def copy_files(file_list, src_img_dir, src_lbl_dir, dst_img_dir, dst_lbl_dir):
    """
    Copy image and label files to destination directories
    """
    os.makedirs(dst_img_dir, exist_ok=True)
    os.makedirs(dst_lbl_dir, exist_ok=True)
    
    for fname in file_list:
        # Copy image
        shutil.copy2(os.path.join(src_img_dir, fname), os.path.join(dst_img_dir, fname))
        
        # Copy corresponding label
        label = os.path.splitext(fname)[0] + ".txt"
        lbl_src = os.path.join(src_lbl_dir, label)
        if os.path.exists(lbl_src):
            shutil.copy2(lbl_src, os.path.join(dst_lbl_dir, label))

def print_split_summary(class_name, train, val, test):
    """
    Print summary of the split for a class
    """
    train_orig = len([f for f in train if not re.search(r'_[CD]\d+_aug\d*\.', f)])
    train_aug = len(train) - train_orig
    
    val_orig = len([f for f in val if not re.search(r'_[CD]\d+_aug\d*\.', f)])
    val_aug = len(val) - val_orig
    
    test_orig = len([f for f in test if not re.search(r'_[CD]\d+_aug\d*\.', f)])
    test_aug = len(test) - test_orig
    
    print(f"{class_name}\tTrain\t{train_orig}\t{train_aug}\t{len(train)}")
    print(f"{class_name}\tVal\t{val_orig}\t{val_aug}\t{len(val)}")
    print(f"{class_name}\tTest\t{test_orig}\t{test_aug}\t{len(test)}")
    print()

# === PATHS ===
input_image_dir = r'C:/Users/flexi/DeepL/processed_dataset/images'
input_label_dir = r'C:/Users/flexi/DeepL/processed_dataset/labels'
output_base_dir = r"C:/Users/flexi/DeepL/final_split_dataset"

# === SPLIT CONFIGURATION ===
# Crack split configuration
crack_split = {
    'train_orig': 238,
    'train_aug': 64,
    'val_orig': 68,
    'val_aug': 18,
    'test_orig': 34,
    'test_aug': 10
}

# Delamination split configuration
delam_split = {
    'train_orig': 104,
    'train_aug': 198,
    'val_orig': 30,
    'val_aug': 56,
    'test_orig': 14,
    'test_aug': 30
}

# === PROCESS CRACK IMAGES ===
print("Processing Crack images...")
crack_images = find_class_images(input_image_dir, 'C')
crack_orig, crack_aug = separate_original_augmented(crack_images)

print(f"Found {len(crack_orig)} original and {len(crack_aug)} augmented Crack images")

crack_train, crack_val, crack_test = split_dataset(
    crack_orig, crack_aug,
    crack_split['train_orig'], crack_split['train_aug'],
    crack_split['val_orig'], crack_split['val_aug'],
    crack_split['test_orig'], crack_split['test_aug']
)

# === PROCESS DELAMINATION IMAGES ===
print("Processing Delamination images...")
delam_images = find_class_images(input_image_dir, 'D')
delam_orig, delam_aug = separate_original_augmented(delam_images)

print(f"Found {len(delam_orig)} original and {len(delam_aug)} augmented Delamination images")

delam_train, delam_val, delam_test = split_dataset(
    delam_orig, delam_aug,
    delam_split['train_orig'], delam_split['train_aug'],
    delam_split['val_orig'], delam_split['val_aug'],
    delam_split['test_orig'], delam_split['test_aug']
)

# === COPY FILES TO DESTINATION ===
print("Copying files...")

# Crack files
copy_files(crack_train, input_image_dir, input_label_dir,
           os.path.join(output_base_dir, 'train', 'images'),
           os.path.join(output_base_dir, 'train', 'labels'))

copy_files(crack_val, input_image_dir, input_label_dir,
           os.path.join(output_base_dir, 'val', 'images'),
           os.path.join(output_base_dir, 'val', 'labels'))

copy_files(crack_test, input_image_dir, input_label_dir,
           os.path.join(output_base_dir, 'test', 'images'),
           os.path.join(output_base_dir, 'test', 'labels'))

# Delamination files
copy_files(delam_train, input_image_dir, input_label_dir,
           os.path.join(output_base_dir, 'train', 'images'),
           os.path.join(output_base_dir, 'train', 'labels'))

copy_files(delam_val, input_image_dir, input_label_dir,
           os.path.join(output_base_dir, 'val', 'images'),
           os.path.join(output_base_dir, 'val', 'labels'))

copy_files(delam_test, input_image_dir, input_label_dir,
           os.path.join(output_base_dir, 'test', 'images'),
           os.path.join(output_base_dir, 'test', 'labels'))

# === PRINT SUMMARY ===
print("\nDataset split summary:")
print("Split\tClass\tOrig\tAug\tTotal")
print_split_summary("Crack", crack_train, crack_val, crack_test)
print_split_summary("Delamination", delam_train, delam_val, delam_test)

print("\n✅ Dataset split completed according to specified table!")

Processing Crack images...
Found 340 original and 92 augmented Crack images
Processing Delamination images...
Found 148 original and 284 augmented Delamination images
Copying files...

Dataset split summary:
Split	Class	Orig	Aug	Total
Crack	Train	238	64	302
Crack	Val	68	18	86
Crack	Test	34	10	44

Delamination	Train	104	198	302
Delamination	Val	30	56	86
Delamination	Test	14	30	44


✅ Dataset split completed according to specified table!
